<a href="https://colab.research.google.com/github/Tahira2910/NLP/blob/main/Text_Classification_Model_using_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim -q
!pip install matplotlib -q

In [ ]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string

In [ ]:
gensim.__version__

In [ ]:
np.random.seed(42)

In [ ]:
# using gensim api, we can download pre trined wordvectors
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

In [ ]:
# loading vector
# wv = api.load('glove-twitter-50')

In [ ]:
# loading vector
# wv = api.load('word2vec-google-news-300')
# wv.save('/content/drive/MyDrive/Vectors.kv')

In [ ]:
wv['apple']

In [ ]:
len(wv['apple'])

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
wv = KeyedVectors.load('/content/drive/MyDrive/Vectors.kv')

In [ ]:
wv.similarity('apple', 'mango')

In [ ]:
wv.similarity('apple', 'car')

In [ ]:
# checking similarity pairwise
pairs = [
    ('car', 'minivan'),
    ('car', 'bicycle'),
    ('car', 'airplane'),
    ('car', 'cereal'),
    ('car', 'communism')
]
for w1, w2 in pairs:
  print('%r\t%r\t%.2f' % (w1,w2, wv.similarity(w1, w2)))

In [ ]:
# to get  5 similar words to those specified in this list, with a score associated with it.
print(wv.most_similar(positive = ['car', 'minivan'], topn = 5))

In [ ]:
print(wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car']))

**Semantic regularities captured in word embeddings**

In [ ]:
wv.most_similar(positive = ['woman', 'king'], negative = ['man'], topn = 3)

In [ ]:
words = ['one', 'two', 'man', 'woman', 'table']

In [ ]:
# https://deepai.org/dataset/text8

In [ ]:
sample_vectors = np.array([wv[word] for word in words])
# PCA: Principal Component Analysis-->reduces those 300 dimensions into 2 dim
#  so that it is able to visualize
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
result = pca.fit_transform(sample_vectors)
result

In [ ]:
plt.figure(figsize = (12,12))
plt.scatter(result[:, 0], result[:, 1])
for i, word in enumerate(words):
  plt.annotate(word, xy = (result[i, 0], result[i,1]))
  plt.show()

In [ ]:
# all the setences must have same length, for that we will take average...

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("/content/train.csv", error_bad_lines=False, engine="python")

In [ ]:
data = data[:500]
data.toxic.value_counts()

In [ ]:
data.head(500)

In [ ]:
import string
import spacy

In [ ]:
punctuations = string.punctuation
print(punctuations)

In [ ]:
nlp = spacy.load("en_core_web_sm")  # spacy small(sm) model
stop_words = nlp.Defaults.stop_words
print(stop_words)


In [ ]:
def sent_vec(sent):
  vector_size = wv.vector_size   # value/dimension: 300
  wv_res = np.zeros(vector_size)    # all vecotrs will be zeros
  # print(wv_res)
  ctr = 1
  for w in sent:    # iterate through our sentence--> w
    if w in wv:    # do we have that particular word present in our word2vec
      ctr += 1    # we will count them. bcz for average we need to know how many in our sentenceS
      wv_res += wv[w]    #  add vector representation of this particular word to zero vector
  wv_res =  wv_res/ctr   # evetually added all the vectors associated with each word in the sentence(of the dataset) in
  # this final vector--> wv_res....and then divide by the no of words-> ctr
  return wv_res


In [ ]:
# Creating tokenizer function
def spacy_tokenizer(sentence):
    #  Creating our token object, which is used to create documents with
    # linguistic annotations.

    doc = nlp(sentence)    # passing the text to nlp function(spacy model)
  # print(doc), it will give tokens(word, symbol in eng language)
  # doc--> object
    # print(doc)
    # print(type(doc))
    # Lemmatizing each token and converting each token into lowercase
    # lemma--> root word for that particular english word, then make it lowercase and then strip left or right space if it has
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

    # return preprocessed list of tokens
    return mytokens



In [ ]:
sent_vec('I am happy')

In [ ]:
data['tokens'] = data['comment_text'].apply(spacy_tokenizer)

In [ ]:
data.head()

In [ ]:
# to get avg vectors
data['vec'] = data['tokens'].apply(sent_vec)

In [ ]:
data.head()

In [ ]:
X = data['vec'].to_list()
Y = data['toxic'].to_list()

In [ ]:
#output: 300 dim vector
X[0]

In [ ]:
from sklearn.model_selection import train_test_split
# stratify=ylabels--> to maintain the balance of toxic and non toxic data of y
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)


In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
from sklearn import metrics

In [ ]:
predicted = classifier.predict(X_test)
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:", metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:", metrics.recall_score(y_test, predicted))
